yfinance API Documentation: https://ranaroussi.github.io/yfinance/reference/index.html 

In [3]:
import sys 
import os

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

import importlib

# Add root path so other subfolders are accessible
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


import src.utils.db_azure
# Reload it every time the cell is run
importlib.reload(src.utils.db_azure)

from src.utils.db_azure import get_analytics_azure_engine
from src.utils.db_azure import get_dashboard_azure_engine
from src.utils.db_azure import azure_upsert



In [23]:
# get some stock data
ticker = "BIR.TO"
# data = yf.download(ticker, start="2025-01-01", end="2025-09-25")
data = yf.download(ticker, period ='60d', interval= '5m', group_by='ticker')

# Clean up multi-index column names to just be the feature names (close, open etc) & make lowercase
data = data.stack(level=0,future_stack=True).rename_axis(['Date', 'Ticker']).reset_index(level=1)

# remote date as the index
data = data.reset_index()

data.columns = [col.lower() for col in data.columns]

# add current timestamp for future QC
data['updated_at'] = pd.Timestamp.now()




C:\Users\tomwm\AppData\Local\Temp\ipykernel_8136\3696860929.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period ='60d', interval= '5m', group_by='ticker')
[*********************100%***********************]  1 of 1 completed


In [ ]:
# Create new table and load to sql
pgs_engine = get_pgs_engine()

data.to_sql(
    'stock_prices',  
    pgs_engine,
    if_exists='fail',   
    index=False       
)

640

In [11]:
from sqlalchemy.dialects.mssql import DATETIME2, VARCHAR, FLOAT, BIGINT

# Create new table and load to sql
azure_engine = get_analytics_azure_engine()


dtype_mapping = {
    'date': DATETIME2(6),        # precise for milliseconds
    'ticker': VARCHAR(20),       
    'open': FLOAT,
    'high': FLOAT,
    'low': FLOAT,
    'close': FLOAT,
    'volume': BIGINT,            
    'updated_at': DATETIME2(0)   # drops seconds, rounds to nearest minute
}

data.to_sql(
    'stock_prices',
    azure_engine,
    if_exists='fail', 
    index=False,
    dtype=dtype_mapping
)

165

In [14]:
# Create indices on new table
from sqlalchemy import text

with azure_engine.begin() as conn:
    conn.execute(text('alter table stock_prices alter column date datetime2 NOT NULL'))
    conn.execute(text('alter table stock_prices alter column ticker varchar(20) NOT NULL'))
    conn.execute(text('alter table stock_prices add constraint pkey_stock_prices PRIMARY KEY (date,ticker)'))

In [24]:
# upsert dataframe to postgres
azure_engine = get_analytics_azure_engine()
azure_upsert(data,azure_engine,'stock_prices')

In [28]:
# get data from postgres db
azure_engine = get_analytics_azure_engine()
select_query = f"select * from stock_prices where ticker = '{ticker}'"

df = pd.read_sql(select_query,azure_engine)

# convert time to EST
# df['date'] = df['date'].dt.tz_convert('America/Toronto')


df.head()

,date,ticker,open,high,low,close,volume,updated_at
0,2025-07-07 13:30:00,BIR.TO,7.05,7.07,6.960,6.980,0,2025-09-30 15:22:13
1,2025-07-07 13:35:00,BIR.TO,6.97,6.97,6.940,6.955,11320,2025-09-30 15:22:13
2,2025-07-07 13:40:00,BIR.TO,6.97,6.97,6.930,6.930,8200,2025-09-30 15:22:13
3,2025-07-07 13:45:00,BIR.TO,6.93,6.95,6.930,6.940,6900,2025-09-30 15:22:13
4,2025-07-07 13:50:00,BIR.TO,6.94,7.02,6.925,6.975,36500,2025-09-30 15:22:13


In [29]:
# visualize prices
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = df['date'], y = df['open'],
    name = 'price',
    yaxis = 'y1',
    mode = 'lines+markers'

))


# Layout with multiple y-axes
fig.update_layout(
    title='Da Title',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Series 1', side='left'),
    legend=dict(x=0.01, y=0.99)
)

fig.show()
